### Install the prerequisite.

In [1]:
!pip install deepsparse[yolo]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 94.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 KB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 1.8 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.14.1+cu116
    

### Install modoptima from the source

In [2]:
#Clone the repo.
!git clone https://github.com/VikasOjha666/modoptima.git
#Changing directory to the cloned.
%cd modoptima
#Build and install.
!python setup.py install
%cd ../
#Remove the cloned folder.
!rm -rf modoptima

Cloning into 'modoptima'...
remote: Enumerating objects: 212, done.
remote: Counting objects: 100% (212/212), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 212 (delta 74), reused 208 (delta 70), pack-reused 0
Receiving objects: 100% (212/212), 33.57 MiB | 17.10 MiB/s, done.
Resolving deltas: 100% (74/74), done.
/content/modoptima
running install
running bdist_egg
running egg_info
creating modoptima.egg-info
writing modoptima.egg-info/PKG-INFO
writing dependency_links to modoptima.egg-info/dependency_links.txt
writing requirements to modoptima.egg-info/requires.txt
writing top-level names to modoptima.egg-info/top_level.txt
writing manifest file 'modoptima.egg-info/SOURCES.txt'
package init file 'modoptima/__init__.py' not found (or not a regular file)
writing manifest file 'modoptima.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/modoptima
co

### Download and install stop sign dataset.

In [3]:
!gdown --id 1j4w1UXzCTkzkiKnwj1Luo1x0ie9Yse_b

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1j4w1UXzCTkzkiKnwj1Luo1x0ie9Yse_b
To: /content/stop_sign_dataset_yolov7.zip
100% 19.8M/19.8M [00:00<00:00, 95.9MB/s]


In [4]:
!unzip /content/stop_sign_dataset_yolov7.zip

Archive:  /content/stop_sign_dataset_yolov7.zip
 extracting: README.roboflow.txt     
 extracting: data.yaml               
   creating: train/
   creating: train/images/
 extracting: train/images/100_jpg.rf.115423eaad5395c8601884eed2ada5aa.jpg  
 extracting: train/images/100_jpg.rf.194be21decf7c42daa4953dc405ef058.jpg  
 extracting: train/images/100_jpg.rf.bbd52ac7f66fa9d0257b468194340db8.jpg  
 extracting: train/images/101_jpg.rf.636d5d5efd5430193a2884a82f9cd8d6.jpg  
 extracting: train/images/101_jpg.rf.667bd1f34722e629c6a82d7c20c33025.jpg  
 extracting: train/images/101_jpg.rf.70f647939a526a6d6825566e4f605116.jpg  
 extracting: train/images/102_jpg.rf.1960e83ca1f6422ccd0593413126c041.jpg  
 extracting: train/images/102_jpg.rf.b2ed75cc599aa9ff5d4525dca11dd833.jpg  
 extracting: train/images/102_jpg.rf.ee234980138898c1818e647ecfd38c27.jpg  
 extracting: train/images/103_jpg.rf.0464471a861f5d6fef87ccaf93a7cead.jpg  
 extracting: train/images/103_jpg.rf.d4c936acdf1931fddad67264ac19772e

### Edit the data yaml file to point to train and test img folders.

In [5]:
%%writefile data.yaml
train: ./train/images
val: ./valid/images
test: ./test/images

nc: 1
names: ['stop']

Overwriting data.yaml


### Importing the YOLOV7 trainer class from modoptima

In [1]:
from modoptima.MasterTrainer import YOLOV7

Import onnx_graphsurgeon failure: No module named 'onnx_graphsurgeon'


In [4]:
!rm -rf /content/optmodel

### Creating the pruned and quantized tiny YOLO V7 model.

##### Here we are creating a pruned and quantized model for my stop sign datasets. The <i> epoch param </i> is used for setting the no of epochs for which we need to train the model,<i> data </i> param is used to specify the data yaml of the dataset in the same way as we do for training the yolo v7 model using official code,<i> weights </i> param in this case is set to weights meaning we want to initialize our weights from scratch. In case we want to resume training we can set the <i> weights </i> params to the model path. In case we want to do transfer learning from a already trained modoptima model we need to specify the path to the pt file and set transfer_learning params to True. The param <i> notest </i> is used to specify whether we want to test the model at each epoch in training,<i> quantization </i> param is set to true is we want to quantize the model. Setting it to false will just create a pruned model,<i>prun_start_epoch </i> param is used to specify the epoch from which pruning should begin and <i> prun_end_epoch </i> specify the epoch upto which the pruning should be continued. The <i> quant_start_epoch </i> is used to specify the epoch at which the quantization should begin.

In [5]:
tiny_yolo=YOLOV7(batch_size=32
                                     ,epochs=150,
                                     data='/content/data.yaml',
                                     weights='weights',
                                     device='cuda:0',
                                     notest=False,quantize=True,prun_start_epoch=0,prun_end_epoch=90,quant_start_ep=97)
#In case of transfer learning.

# tiny_yolo=YOLOV7(batch_size=32
#                                      ,epochs=100,
#                                      data='/content/data.yaml',
#                                      weights='/modelpath/model.pt',
#                                      device='cuda:0',
#                                      notest=False,quantize=True,prun_start_epoch=0,prun_end_epoch=90,quant_start_ep=97,transfer_learning=True)


#For resuming the training from some epoch.

# tiny_yolo=YOLOV7(batch_size=32
#                                      ,epochs=100,
#                                      data='/content/data.yaml',
#                                      weights='/modelpath/ckpt.pt',
#                                      device='cuda:0',
#                                      notest=False,quantize=True,prun_start_epoch=0,prun_end_epoch=90,quant_start_ep=97)




### Calling this function to create the pruned and quantized model.

In [6]:
tiny_yolo.create_optimized_model()

github: skipping check (not a git repository)
requirements: /content/requirements.txt not found, check failed.
wandb: Install Weights & Biases for YOLOR logging with 'pip install wandb' (recommended)


train: Scanning 'train/labels.cache' images and labels... 408 found, 0 missing, 0 empty, 0 corrupted: 100%|██████████| 408/408 [00:00<?, ?it/s]
val: Scanning 'valid/labels.cache' images and labels... 15 found, 0 missing, 0 empty, 0 corrupted: 100%|██████████| 15/15 [00:00<?, ?it/s]



autoanchor: Analyzing anchors... anchors/target = 4.48, Best Possible Recall (BPR) = 1.0000


     0/149     7.81G   0.07548   0.01486         0   0.09034        48       640: 100%|██████████| 13/13 [00:21<00:00,  1.63s/it]


Testing with YOLOv5 AP metric...


...

                 all          15          15           1           1       0.995       0.766


   149/149     14.2G   0.03006  0.005609         0   0.03567        58       640: 100%|██████████| 13/13 [00:21<00:00,  1.68s/it]


Testing with YOLOv5 AP metric...


               Class      Images      Labels           P           R      mAP@.5  mAP@.5:.95: 100%|██████████| 1/1 [00:00<00:00,  1.61it/s]


                 all          15          15           1           1       0.995       0.791


### Exporting model for inference with size 512x512.(Size could be anything depending on the need and given that original YOLO model supports.)

In [7]:
tiny_yolo.export_model("/content/optmodel/exp/weights/best.pt",img_size=[512,512])


Starting ONNX export with onnx 1.12.0...


/usr/local/lib/python3.8/dist-packages/torch/onnx/utils.py:439: UserWarning: It is recommended that constant folding be turned off ('do_constant_folding=False') when exporting the model in training-amenable mode, i.e. with 'training=TrainingMode.TRAIN' or 'training=TrainingMode.PRESERVE' (when model is in training mode). Otherwise, some learnable model parameters may not translate correctly in the exported ONNX model because constant folding mutates model parameters. Please consider turning off constant folding or setting the training=TrainingMode.EVAL.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/modoptima-0.1.0-py3.8.egg/modoptima/Train/yolov7tiny/models/yolo.py:582: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if augment:
/usr/local/lib/python3.8/dist-pa

ONNX export success, saved as /content/optmodel/exp/weights/best.onnx
CoreML export failure: No module named 'coremltools'

Export complete (39.27s).
